In [2]:
import sys, os
from pathlib import Path
import pickle
import multiprocessing as mp
import multiprocessing.shared_memory as shm

import numpy as np
import pandas as pd

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from nomad import costs
from nomad import conf
from nomad import shortest_path as sp

In [3]:
# Get params for MAC POSTS tdsp function
tdsp_folder = Path().resolve() / 'tdsp_files_sn'

In [4]:
# Read in the supernetwork as an object
graph_sn_path = Path().resolve() / 'graphs' / 'graph_sn.pkl'
with open(graph_sn_path, 'rb') as inp:
    G_sn = pickle.load(inp)
df_G = costs.edges.nx_to_df(G_sn).sort_values(by=['source','target','mode_type']).reset_index(drop=True)

### Create graph file and get node/link IDs for subsequent use
df_G = sp.prepare_graph_file(tdsp_folder, G_sn)
nid_map = sp.get_nid_map(df_G)
inv_nid_map = dict(zip(nid_map.values(), nid_map.keys()))
linkID_map = sp.get_link_id_map(df_G)
inv_linkID_map = dict(zip(linkID_map.values(), linkID_map.keys()))
linkID_arr = df_G['linkID'].to_numpy().reshape((-1,1))

In [5]:
# Get all time-dep edge costs
df_tt_dynamic, df_rel_dynamic, df_price_dynamic, df_risk_dynamic, df_disc_dynamic = costs.edges.dynamic.assign_edge_costs(G_sn)

In [6]:
# get the indices of scooter traversal links
sc_trav_links = [link_id for named_link, link_id in inv_linkID_map.items() if named_link[0].startswith('sc') and named_link[1].startswith('sc')]

In [7]:
# Parameters
NUM_INTERVALS = conf.NUM_INTERVALS  
interval_columns = [f'i{i}' for i in range(NUM_INTERVALS)]

### Prepare link cost files for different VOTs
# beta weighting factors
vot = 10
BETAS = {
    'tt': vot/3600,
    'rel': 10/3600,
    'x': 1,
    'risk': 20,
    'disc': 0}

gtc_arr_no_price = BETAS['rel'] * df_rel_dynamic[interval_columns].values.astype(np.float16) + BETAS['tt'] * df_tt_dynamic[interval_columns].values.astype(np.float16) + BETAS['risk'] * df_risk_dynamic[interval_columns].values.astype(np.float16) + BETAS['disc'] * df_disc_dynamic[interval_columns].values.astype(np.float16)

In [8]:
for sc_ppmin in [0.09, 0.14, 0.19, 0.24, 0.29, 0.34]:
    price_reduction_pct = (0.39 - sc_ppmin) / 0.39  # percent reduction in scoot link cost
    price_arr = df_price_dynamic[interval_columns].values.astype(np.float16).copy() 
    price_arr[sc_trav_links,:] *= (1-price_reduction_pct) 
    gtc_arr = gtc_arr_no_price + price_arr
    linkID_arr = df_G['linkID'].to_numpy().reshape((-1,1))
    filename = 'td_link_cost_sc' + str(sc_ppmin)
    sp.prepare_gtc_file(tdsp_folder, filename, linkID_arr, gtc_arr)